Importing all necessary packages to manipulate the data

In [1]:
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy

In [2]:
originalData=pd.read_csv('./CrimeData.csv')
originalData.tail(5)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,False,...,0::25,0::Adult 18+,0::Female,0::Jhkeya Tezeno,NaN,"0::Unharmed, Arrested",0::Subject-Suspect,http://www.klfy.com/news/local/rayne-woman-cha...,NaN,NaN
239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,http://www.ksla.com/story/37854648/man-wanted-...,False,...,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Jamal Haskett||1::Jaquarious Tyjuan Ardison,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://www.ksla.com/story/37854648/man-wanted-...,23.0,31.0
239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/sh...,False,...,0::21,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.nola.com/crime/index.ssf/2018/04/sh...,85.0,7.0
239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,https://www.chron.com/news/houston-texas/houst...,False,...,0::42,0::Adult 18+,0::Male,0::Leroy Ellis,NaN,0::Killed,0::Victim,http://www.khou.com/article/news/hpd-investiga...,149.0,17.0
239676,1081940,2018-03-31,Maine,Norridgewock,434 Skowhegan Rd,2,0,http://www.gunviolencearchive.org/incident/108...,https://www.centralmaine.com/2018/03/31/police...,False,...,0::58||1::62,0::Adult 18+||1::Adult 18+,0::Female||1::Male,0::Marie Lancaster Hale||1::William Hale,1::Significant others - current or former,0::Killed||1::Killed,0::Victim||1::Subject-Suspect,https://www.centralmaine.com/2018/03/31/police...,111.0,3.0


In [3]:
originalData[['congressional_district', 'gun_stolen',
       'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',]].tail(50)

,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes
239627,4.0,0::Unknown,0::Unknown,Shot - Wounded/Injured,41.1671,NaN,-73.1994,1.0,"shot in side, leg"
239628,4.0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)",30.3144,NaN,-81.7793,1.0,NaN
239629,1.0,0::Unknown,0::Unknown,Institution/Group/Business||Non-Shooting Incid...,42.0925,Pearl House Chinese Restaurant,-72.5803,1.0,pw employee during ar
239630,13.0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)||Dri...",42.3301,NaN,-83.1486,1.0,NaN
239631,NaN,0::Unknown,0::Unknown,Shots Fired - No Injuries||ShotSpotter,NaN,NaN,NaN,1.0,shotspotter
239632,12.0,0::Unknown,0::Unknown,Shot - Wounded/Injured||Institution/Group/Busi...,35.1372,B & B Mart,-80.8772,1.0,NaN
239633,NaN,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)||Off...",NaN,NaN,NaN,1.0,NaN
239634,29.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.6872,Emerald Pointe complex,-95.2024,1.0,NaN
239635,2.0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)",30.4490,NaN,-84.3068,1.0,NaN
239636,7.0,0::Unknown,0::Unknown,Shot - Wounded/Injured,39.8850,The Villas at Meridian Hills,-86.1967,1.0,"Man critical, woman shot, hand, stable;"


In [4]:
originalData.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'address', 'n_killed',
       'n_injured', 'incident_url', 'source_url',
       'incident_url_fields_missing', 'congressional_district', 'gun_stolen',
       'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district'],
      dtype='object')

In [5]:
originalData.describe()

,incident_id,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district
count,2.396770e+05,239677.000000,239677.000000,227733.000000,231754.000000,231754.000000,140226.000000,200905.000000,207342.00000
mean,5.593343e+05,0.252290,0.494007,8.001265,37.546598,-89.338348,1.372442,55.447132,20.47711
std,2.931287e+05,0.521779,0.729952,8.480835,5.130763,14.359546,4.678202,42.048117,14.20456
min,9.211400e+04,0.000000,0.000000,0.000000,19.111400,-171.429000,1.000000,1.000000,1.00000
25%,3.085450e+05,0.000000,0.000000,2.000000,33.903400,-94.158725,1.000000,21.000000,9.00000
50%,5.435870e+05,0.000000,0.000000,5.000000,38.570600,-86.249600,1.000000,47.000000,19.00000
75%,8.172280e+05,0.000000,1.000000,10.000000,41.437375,-80.048625,1.000000,84.000000,30.00000
max,1.083472e+06,50.000000,53.000000,53.000000,71.336800,97.433100,400.000000,901.000000,94.00000


In [6]:
len(originalData.incident_id)

239677

The total number of crimes in USA where a fire gun was involved from 2013 to 2018 is 239677. In this study we will look into more detail in the numbers to discover the nature and chronologic evolution of crime.

Some of the columns in the data are not relevant for what we need to check so we can get rid of a few columns in  order to have a more handleable dataset

In [7]:
workingData=originalData.drop(['address','gun_stolen', 'gun_type','incident_characteristics','notes','incident_url','participant_age','participant_gender','participant_status','participant_type', 'source_url',
       'incident_url_fields_missing', 'congressional_district', 'latitude',
       'location_description', 'longitude',
       'participant_age_group', 'participant_name',
       'sources', 'state_house_district',
       'state_senate_district'], axis=1)
workingData.head(7)

,incident_id,date,state,city_or_county,n_killed,n_injured,n_guns_involved,participant_relationship
0,461105,2013-01-01,Pennsylvania,Mckeesport,0,4,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,1,3,NaN,NaN
2,478855,2013-01-01,Ohio,Lorain,1,3,2.0,NaN
3,478925,2013-01-05,Colorado,Aurora,4,0,NaN,NaN
4,478959,2013-01-07,North Carolina,Greensboro,2,2,2.0,3::Family
5,478948,2013-01-07,Oklahoma,Tulsa,4,0,NaN,NaN
6,479363,2013-01-19,New Mexico,Albuquerque,5,0,2.0,5::Family


We now need to identify the null values in our dataset, the best way to this is calculating the number of nulls per column and calculating their percentage over the total amount of registers.

In [8]:
null_cols = workingData.isnull().sum()*100/len(workingData.incident_id)
null_cols[null_cols > 0]

n_guns_involved             41.493760
participant_relationship    93.418643
dtype: float64

We can observe that the column "participant_relationship" contains approximately 93% of null values. Due to the nature of the information contained in that column, we can assume that no value means no relationship. Therefore, we will exchange the nulls with the string "False". In the same manner, we are not really interested in the kind of relationship, but in the existance of a relationship, therefore, registers with a value can be exchanged with "True"

In [9]:
# We are fillin NA 
workingData['participant_relationship']=workingData.participant_relationship.fillna(0)
workingData['participant_relationship'].head()

0            0
1            0
2            0
3            0
4    3::Family
Name: participant_relationship, dtype: object

In [10]:
workingData.participant_relationship[workingData.participant_relationship!=0] = 1
workingData.participant_relationship.head()

/home/albertocastellanos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    0
1    0
2    0
3    0
4    1
Name: participant_relationship, dtype: object

From checking the data types, we know that some of the columns are expressed with the wrong data type so we should change it.

In [11]:
workingData.dtypes

incident_id                   int64
date                         object
state                        object
city_or_county               object
n_killed                      int64
n_injured                     int64
n_guns_involved             float64
participant_relationship     object
dtype: object

In [12]:
workingData['participant_relationship']=workingData.participant_relationship.astype('bool')
workingData['participant_relationship'].head(5)

0    False
1    False
2    False
3    False
4     True
Name: participant_relationship, dtype: bool

In [13]:
workingData.date=workingData.date.astype('datetime64[ns]')
workingData.date.head(5)

0   2013-01-01
1   2013-01-01
2   2013-01-01
3   2013-01-05
4   2013-01-07
Name: date, dtype: datetime64[ns]

We know that "n-guns-involved"'s NaN values never equal zero since the current dataset refers to crimes where a fire gun was involved. NaN values simply means that the police was unable to clarify the number of guns present in the crime, so this value could vary quite a lot. Therefore, we have decided that we just want to work with cases where we know the number of guns that were used in the crime. 

In [14]:
workingData=workingData.dropna(subset=['n_guns_involved'])
workingData.head()

,incident_id,date,state,city_or_county,n_killed,n_injured,n_guns_involved,participant_relationship
2,478855,2013-01-01,Ohio,Lorain,1,3,2.0,False
4,478959,2013-01-07,North Carolina,Greensboro,2,2,2.0,True
6,479363,2013-01-19,New Mexico,Albuquerque,5,0,2.0,True
10,491674,2013-01-23,Tennessee,Chattanooga,1,3,1.0,False
11,479413,2013-01-25,Missouri,Saint Louis,1,3,1.0,False


In [15]:
workingData.isnull().sum()

incident_id                 0
date                        0
state                       0
city_or_county              0
n_killed                    0
n_injured                   0
n_guns_involved             0
participant_relationship    0
dtype: int64

We can now be sure that there are not null values in our dataset and therefore we are safe to extract conclusions from it

The name of the columns was not as descriptive as desired so we have decided to rename them as necessary:

In [16]:
workingData.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'n_killed',
       'n_injured', 'n_guns_involved', 'participant_relationship'],
      dtype='object')

In [17]:
workingData=workingData.rename(columns={'city_or_county':'city'})

In [18]:
workingData.columns

Index(['incident_id', 'date', 'state', 'city', 'n_killed', 'n_injured',
       'n_guns_involved', 'participant_relationship'],
      dtype='object')

In [19]:
workingData.columns = ['crimeId', 'date', 'state', 'city', 'numberKilled',
       'numberInjured', ' numberGunsInvolved', 'authorsRelationship']

We have also decided to re-arrange the column order for clarity:

In [20]:
columnOrder = ['crimeId', 'date', 'state', 'city', 'authorsRelationship', 'numberKilled',
       'numberInjured', ' numberGunsInvolved']

Data is ready to be analized so we can now export it to .csv now:

In [22]:
workingData.to_csv('./CrimeData_Cleaned.csv', index=False)

Ranking of 10 states whith higher number of crimes

In [ ]:
workingData['state'].value_counts().head(10)

Ranking of 10 states by number of crimes when killed=1

In [ ]:
Top10for5killed=workingData[(workingData.numberKilled==1)]
Top10for5killed['state'].value_counts().head(10)

In [ ]:
workingData.head(5)

We now want to discover the 10 days where more crimes with victims but no casulties happened in the city of Chicago

In [ ]:
dayRankVictims=workingData[(workingData.city=="Chicago") & (workingData.numberInjured>=1) & (workingData.numberKilled==0)]
dayRankVictims.date.value_counts().head(10)

We are going to categorize each register based on the number of killed people in the event. For that purpose we will use bin in order to establish 4 groups.

In [ ]:
numberOfKills=["Zero to Three", "Four to Six", "Six to Nine", "Ten or more"]
cutoffs=[-1,3,6,9,999]
bins=pd.cut(workingData["numberKilled"],cutoffs,labels=numberOfKills)
print(workingData.columns)
dfbins=bins.to_frame()
dfbins.columns=['bins']
dfbins.bins.value_counts()




We can observe that the number of mass killings is 6, we want now to discover the states where more mass killings happened:

In [ ]:
dataWithBins=pd.concat([workingData, dfbins], axis=1)
dataWithBins

I have been trying to resolve a group by with several columns that aggregates a column and sort by it but I have been unable to achieve any result

dataWithBins.groupby(['bins', 'state']).count().sort_values("numberKilled")

In [ ]:
dataWithBins.loc[dataWithBins['bins'].str.startswith('Ten'), 'Mass Killing']=True
dataWithBins.loc[(dataWithBins['bins'].str.startswith('Ten')==False), 'Mass Killing']=False

The top ten of states with more Mass Killings is showing below:

In [ ]:
MassKillingStates=dataWithBins.groupby(['state']).sum().sort_values('Mass Killing', ascending=False).head(10)
MassKillingStates=MassKillingStates[['Mass Killing']]
MassKillingStates

In [ ]:
numberKillsGroups = pd.get_dummies(dataWithBins['bins'])
numberKillsGroups.head(10)

In [ ]:
workingDataWithDummies=pd.merge(workingData,numberKillsGroups, left_index=True, right_index=True)
workingDataWithDummies.head(10)